In [1]:
import pandas as pd
import preprocessor as p

In [2]:
dataset_name = ' ' 
data = pd.read_csv('./datasets/raw/' + dataset_name + '.csv')

In [3]:
data.count()

username            5
location            3
following           5
followers           5
totaltweets         5
whentweetcreated    5
whenacctcreated     5
retweets            5
text                5
hashtags            5
dtype: int64

In [4]:
# NULL DEĞERLER İÇEREN SATIR VE SÜTUNLARIN SİLİNMESİ VE TEKRAR EDEN SÜTUNLARIN SİLİNMESİ
tweets = data.dropna()
tweets = data.drop_duplicates()

In [5]:
tweets.count()

username            5
location            3
following           5
followers           5
totaltweets         5
whentweetcreated    5
whenacctcreated     5
retweets            5
text                5
hashtags            5
dtype: int64

In [6]:
#cleaning text from urls, hashtags, mentions, emojis
def preprocess_tweet(row):
    text = row['text']
    p.set_options(p.OPT.URL, p.OPT.HASHTAG, p.OPT.SMILEY,
                    p.OPT.MENTION, p.OPT.NUMBER, p.OPT.ESCAPE_CHAR,
                    p.OPT.RESERVED)
    text = p.clean(text)
    return text

In [7]:
#apply the function to text column, axis = 1 for all rows
tweets['text'] = tweets.apply(preprocess_tweet, axis=1)

In [8]:
#removing punctuations and lower text
tweets['text'] = tweets['text'].str.lower().str.replace('[^\w\s]', ' ').str.replace('\s\s+', ' ')

In [9]:
tweets.head()

,username,location,following,followers,totaltweets,whentweetcreated,whenacctcreated,retweets,text,hashtags
0,ComicGuySteve75,Northern California,203,221,11773,2021-03-26 22:19:30,2020-04-02 17:10:23,0,tell your barber you want the,"[{'text': 'SnyderCut', 'indices': [45, 55]}]"
1,NaskarPriyajit,NaN,7,5,692,2021-03-26 22:19:26,2020-08-31 11:13:08,1,if canada s crave got mil then imagine what th...,"[{'text': 'SnyderCut', 'indices': [85, 95]}]"
2,MarioHtter1,NaN,31,19,8817,2021-03-26 22:19:24,2020-12-31 13:00:22,91,to the lean and mean trending machine thank yo...,"[{'text': 'SnyderCut', 'indices': [89, 99]}, {..."
3,KungfuScubaBAPE,"Los Angeles, CA",202,207,5101,2021-03-26 22:19:23,2011-05-25 13:09:26,1488,half man half machine all heart snyder s justi...,"[{'text': 'Cyborg', 'indices': [50, 57]}, {'te..."
4,jelani82,"Miami, FL",1597,898,34090,2021-03-26 22:19:22,2010-01-07 23:11:04,1,most of my facebook friends are mcu fans and d...,"[{'text': 'ZackSnydersJusticeLeague', 'indices..."


In [11]:
tweets.to_csv("./datasets/cleaned/" + dataset_name + ".csv")